In [2]:
 from selenium import webdriver

In [3]:
from xlrd import open_workbook
from xlwt import Workbook

In [4]:
from xlutils.copy import copy
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException
import os

In [5]:
driver = webdriver.Chrome()

In [6]:
driver.get("http://www.juso.go.kr/support/AddressMainSearch.do?searchType=TOTAL")

In [7]:
import pandas

In [8]:
wd=pandas.read_excel("크롤링.xlsx")

In [9]:
wd.head()

,도로명칭,법정동,주소2,x.y,y.y,아파트_단지_수.y,아파트_면적_66_제곱미터_미만_세대_수.y,아파트_면적_66_제곱미터_세대_수.y,아파트_면적_99_제곱미터_세대_수.y,아파트_면적_132_제곱미터_세대_수.y,아파트_면적_165_제곱미터_세대_수.y,아파트_가격_1_억_미만_세대_수.y,아파트_가격_1_억_세대_수.y,아파트_가격_2_억_세대_수.y,아파트_가격_3_억_세대_수.y,아파트_가격_4_억_세대_수.y,아파트_가격_5_억_세대_수.y,아파트_가격_6_억_이상_세대_수.y,아파트_평균_면적.y,아파트_평균_시가.y
0,가로공원로,NaN,서울특별시 양천구 가로공원로 106,126.8277,37.53554,1164,9190.0,2922.0,NaN,NaN,NaN,6022.0,5002.0,NaN,NaN,NaN,NaN,NaN,202.166374,385094049.8
1,가로공원로,NaN,서울특별시 강서구 가로공원로 178,126.8357,37.53667,1164,9190.0,2922.0,NaN,NaN,NaN,6022.0,5002.0,NaN,NaN,NaN,NaN,NaN,202.166374,385094049.8
2,가로공원로,NaN,서울특별시 양천구 가로공원로 133,126.8289,37.53655,1164,9190.0,2922.0,NaN,NaN,NaN,6022.0,5002.0,NaN,NaN,NaN,NaN,NaN,202.166374,385094049.8
3,가로공원로,NaN,서울특별시 강서구 가로공원로 221-7,126.8402,37.53831,1164,9190.0,2922.0,NaN,NaN,NaN,6022.0,5002.0,NaN,NaN,NaN,NaN,NaN,202.166374,385094049.8
4,가로공원로,NaN,서울특별시 강서구 가로공원로 200-16,126.8384,37.53652,1164,9190.0,2922.0,NaN,NaN,NaN,6022.0,5002.0,NaN,NaN,NaN,NaN,NaN,202.166374,385094049.8


In [20]:
temp = wd
import time

for i in range(799,2500):
    driver.get("http://www.juso.go.kr/support/AddressMainSearch.do?searchType=TOTAL")
    #엑셀파일열기
    juso = temp['주소2'][i] #검색키워드
    search = driver.find_element_by_id("keyword")
    search.send_keys(juso)
    button = driver.find_element_by_id("searchButton") #검색버튼
    button.click()

    if  "more_info1" in driver.page_source:
        try: 
            if "fixed apt_info" not in driver.page_source:
                more_button = driver.find_element_by_xpath("//*[@id='list1']/td[2]/ul/li[1]/div[2]/span/a[3]")
                driver.execute_script("arguments[0].click();", more_button) #더보기버튼
                box = driver.find_element_by_xpath("//*[@id='more_info1']/td/ul/li[2]") #관할~전번
                temp.loc[i,'법정동'] = box.find_element_by_tag_name('div').text
                time.sleep(1)
            else:
                more_button = driver.find_element_by_xpath("//*[@id='list1']/td[2]/ul/li[1]/div[3]/span/a[3]")
                driver.execute_script("arguments[0].click();", more_button) #더보기버튼
                box = driver.find_element_by_xpath("//*[@id='more_info1']/td/ul/li[2]") #관할~전번
                temp.loc[i,'법정동'] = box.find_element_by_tag_name('div').text
                time.sleep(1)
        except NoSuchElementException:
            temp.loc[i,'법정동'] = "없음"
            time.sleep(1)
            continue


                                 
    
    

In [22]:
temp.loc[0:2500]

,도로명칭,법정동,주소2,x.y,y.y,아파트_단지_수.y,아파트_면적_66_제곱미터_미만_세대_수.y,아파트_면적_66_제곱미터_세대_수.y,아파트_면적_99_제곱미터_세대_수.y,아파트_면적_132_제곱미터_세대_수.y,아파트_면적_165_제곱미터_세대_수.y,아파트_가격_1_억_미만_세대_수.y,아파트_가격_1_억_세대_수.y,아파트_가격_2_억_세대_수.y,아파트_가격_3_억_세대_수.y,아파트_가격_4_억_세대_수.y,아파트_가격_5_억_세대_수.y,아파트_가격_6_억_이상_세대_수.y,아파트_평균_면적.y,아파트_평균_시가.y
0,가로공원로,※관할주민센터 정보는 거주지에 한해서 제공됩니다.,서울특별시 양천구 가로공원로 106,126.8277,37.53554,1164,9190.0,2922.0,NaN,NaN,NaN,6022.0,5002.0,NaN,NaN,NaN,NaN,NaN,202.166374,385094049.8
1,가로공원로,서울특별시 강서구 화곡제1동☎ 02-2600-7525,서울특별시 강서구 가로공원로 178,126.8357,37.53667,1164,9190.0,2922.0,NaN,NaN,NaN,6022.0,5002.0,NaN,NaN,NaN,NaN,NaN,202.166374,385094049.8
2,가로공원로,※관할주민센터 정보는 거주지에 한해서 제공됩니다.,서울특별시 양천구 가로공원로 133,126.8289,37.53655,1164,9190.0,2922.0,NaN,NaN,NaN,6022.0,5002.0,NaN,NaN,NaN,NaN,NaN,202.166374,385094049.8
3,가로공원로,서울특별시 강서구 화곡제1동☎ 02-2600-7525,서울특별시 강서구 가로공원로 221-7,126.8402,37.53831,1164,9190.0,2922.0,NaN,NaN,NaN,6022.0,5002.0,NaN,NaN,NaN,NaN,NaN,202.166374,385094049.8
4,가로공원로,서울특별시 강서구 화곡제1동☎ 02-2600-7525,서울특별시 강서구 가로공원로 200-16,126.8384,37.53652,1164,9190.0,2922.0,NaN,NaN,NaN,6022.0,5002.0,NaN,NaN,NaN,NaN,NaN,202.166374,385094049.8
5,가로공원로,※관할주민센터 정보는 거주지에 한해서 제공됩니다.,서울특별시 양천구 가로공원로 96,126.8265,37.53542,1164,9190.0,2922.0,NaN,NaN,NaN,6022.0,5002.0,NaN,NaN,NaN,NaN,NaN,202.166374,385094049.8
6,가로공원로,※관할주민센터 정보는 거주지에 한해서 제공됩니다.,서울특별시 양천구 가로공원로 100,126.8270,37.53549,1164,9190.0,2922.0,NaN,NaN,NaN,6022.0,5002.0,NaN,NaN,NaN,NaN,NaN,202.166374,385094049.8
7,가로공원로,서울특별시 강서구 화곡제1동☎ 02-2600-7525,서울특별시 강서구 가로공원로 177-4,126.8358,37.53727,1164,9190.0,2922.0,NaN,NaN,NaN,6022.0,5002.0,NaN,NaN,NaN,NaN,NaN,202.166374,385094049.8
8,가로공원로,서울특별시 강서구 화곡제1동☎ 02-2600-7525,서울특별시 강서구 가로공원로 173,126.8353,37.53716,1164,9190.0,2922.0,NaN,NaN,NaN,6022.0,5002.0,NaN,NaN,NaN,NaN,NaN,202.166374,385094049.8
9,가로공원로,서울특별시 강서구 화곡제1동☎ 02-2600-7525,서울특별시 강서구 가로공원로 219-11,126.8399,37.53829,1164,9190.0,2922.0,NaN,NaN,NaN,6022.0,5002.0,NaN,NaN,NaN,NaN,NaN,202.166374,385094049.8


In [24]:
import pandas as pd

In [25]:
temp_df = pd.DataFrame(temp)
#temp_df.loc[0:270]
#temp_df.to_csv("temp.csv", encoding='utf-8')
temp_df.loc[0:2500].to_excel('도로명주소0-2500.xlsx', sheet_name='sheet1')
